### 01_data_collection.ipynb
#### Week 1 - Data Collection (Financial News + Stock Prices)

In [27]:
# check the kernel is in the venv we just created in this project folder
import sys
print(sys.executable)


c:\Users\User\Desktop\Data Science\Fall 2025\Generative AI\Project\genai-financial-doc-analysis\.venv\Scripts\python.exe


In [28]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

print("✅ All packages loaded successfully")

✅ All packages loaded successfully


In [29]:
# ----------------------------
# Setup
# ----------------------------
import pandas as pd
import yfinance as yf
import os

# Create folders if not exist
os.makedirs("../data/raw", exist_ok=True)
os.makedirs("../data/processed", exist_ok=True)

In [30]:
# ----------------------------
# Load Financial News Data
# ----------------------------
# Replace with your actual downloaded CSV path
import pandas as pd

news_path = r"C:\Users\User\Desktop\Data Science\Fall 2025\Generative AI\Project\genai-financial-doc-analysis\data\processed\financial_news_clean.csv"

financial_news = pd.read_csv(news_path, encoding="latin1", header=None, names=["sentiment", "text"])

print("Shape:", financial_news.shape)
print(financial_news.head())


Shape: (4847, 2)
   sentiment                                               text
0  sentiment                                               text
1    neutral  According to Gran , the company has no plans t...
2    neutral  Technopolis plans to develop in stages an area...
3   negative  The international electronic industry company ...
4   positive  With the new production plant the company woul...


In [31]:
# check class inbalance
print(financial_news['sentiment'].value_counts())


sentiment
neutral      2879
positive     1363
negative      604
sentiment       1
Name: count, dtype: int64


In [32]:
# Save a cleam UTF-8 Version for later
financial_news.to_csv("../data/processed/financial_news_clean.csv", index=False, encoding="utf-8")


## STOCK DATA

### 1. Pull stock data from yahoo finance of AAPL (yfinance)

In [33]:
import yfinance as yf
import pandas as pd
import os

# Make sure raw data folder exists
os.makedirs("../data/raw", exist_ok=True)

# Download Apple stock data (daily)
aapl = yf.download("AAPL", start="2020-01-01", end="2025-01-01")

print("AAPL stock data shape:", aapl.shape)
print(aapl.head())

# Save to CSV
aapl.to_csv("../data/raw/AAPL_stock.csv")
print("✅ Saved AAPL stock data to ../data/raw/AAPL_stock.csv")


C:\Users\User\AppData\Local\Temp\ipykernel_18664\3431454893.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  aapl = yf.download("AAPL", start="2020-01-01", end="2025-01-01")
[*********************100%***********************]  1 of 1 completed

AAPL stock data shape: (1258, 5)
Price           Close       High        Low       Open     Volume
Ticker           AAPL       AAPL       AAPL       AAPL       AAPL
Date                                                             
2020-01-02  72.538521  72.598899  71.292311  71.545897  135480400
2020-01-03  71.833282  72.594048  71.608677  71.765659  146322800
2020-01-06  72.405678  72.444321  70.703012  70.954188  118387200
2020-01-07  72.065147  72.671341  71.845369  72.415337  108872000
2020-01-08  73.224411  73.526303  71.768086  71.768086  132079200
✅ Saved AAPL stock data to ../data/raw/AAPL_stock.csv


### Step 2: Organize stock data for multiple companies

In [34]:
tickers = ["AAPL", "MSFT", "GOOG", "AMZN"]

for ticker in tickers:
    df = yf.download(ticker, start="2020-01-01", end="2025-01-01")
    path = f"../data/raw/{ticker}_stock.csv"
    df.to_csv(path)
    print(f"✅ Saved {ticker} stock data to {path}")


C:\Users\User\AppData\Local\Temp\ipykernel_18664\4029043336.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2020-01-01", end="2025-01-01")
[*********************100%***********************]  1 of 1 completed
C:\Users\User\AppData\Local\Temp\ipykernel_18664\4029043336.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2020-01-01", end="2025-01-01")
[*********************100%***********************]  1 of 1 completed
C:\Users\User\AppData\Local\Temp\ipykernel_18664\4029043336.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2020-01-01", end="2025-01-01")
[*********************100%***********************]  1 of 1 completed
C:\Users\User\AppData\Local\Temp\ipykernel_18664\4029043336.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tic

✅ Saved AAPL stock data to ../data/raw/AAPL_stock.csv
✅ Saved MSFT stock data to ../data/raw/MSFT_stock.csv
✅ Saved GOOG stock data to ../data/raw/GOOG_stock.csv


[*********************100%***********************]  1 of 1 completed

✅ Saved AMZN stock data to ../data/raw/AMZN_stock.csv


### Have the 4 stocks in a combined excel file to make it easier to join with news data instead of looping through mulitple stock files.

In [42]:
import yfinance as yf
import pandas as pd

tickers = ["AAPL", "MSFT", "GOOG", "AMZN"]
data = []

for ticker in tickers:
    t = yf.Ticker(ticker)
    df = t.history(period="5y")   # last 5 years
    df = df.reset_index()         # move Date from index to column
    df["Ticker"] = ticker         # add ticker column
    data.append(df)

# Combine all tickers
all_stocks = pd.concat(data, ignore_index=True)

# Save as tidy dataset
all_stocks.to_csv("../data/raw/all_stocks_long.csv", index=False)

print("✅ Long-format dataset saved with shape:", all_stocks.shape)
print(all_stocks.head())


✅ Long-format dataset saved with shape: (5020, 9)
                       Date        Open        High         Low       Close  \
0 2020-09-16 00:00:00-04:00  112.086293  112.835283  108.983321  109.070862   
1 2020-09-17 00:00:00-04:00  106.726601  109.138937  105.744154  107.329681   
2 2020-09-18 00:00:00-04:00  107.388051  107.854952  103.195633  103.925171   
3 2020-09-21 00:00:00-04:00  101.687903  107.183759  100.287187  107.076759   
4 2020-09-22 00:00:00-04:00  109.605857  109.780947  106.181894  108.759590   

      Volume  Dividends  Stock Splits Ticker  
0  154679000        0.0           0.0   AAPL  
1  178011000        0.0           0.0   AAPL  
2  287104900        0.0           0.0   AAPL  
3  195713800        0.0           0.0   AAPL  
4  183055400        0.0           0.0   AAPL  


In [43]:
all_stocks["Date"] = pd.to_datetime(all_stocks["Date"]).dt.date


In [44]:
all_stocks = all_stocks.drop(columns=["Dividends", "Stock Splits"])


In [45]:
all_stocks.head()

,Date,Open,High,Low,Close,Volume,Ticker
0,2020-09-16,112.086293,112.835283,108.983321,109.070862,154679000,AAPL
1,2020-09-17,106.726601,109.138937,105.744154,107.329681,178011000,AAPL
2,2020-09-18,107.388051,107.854952,103.195633,103.925171,287104900,AAPL
3,2020-09-21,101.687903,107.183759,100.287187,107.076759,195713800,AAPL
4,2020-09-22,109.605857,109.780947,106.181894,108.759590,183055400,AAPL


In [46]:
all_stocks.to_csv("../data/processed/all_stocks_clean.csv", index=False)
print("✅ Clean stock dataset saved:", all_stocks.shape)


✅ Clean stock dataset saved: (5020, 7)


#### Quick Validation Code

Run this to confirm your uploaded all_stocks.csv structure:

In [47]:
import pandas as pd

# Load your new file
all_stocks = pd.read_csv(r"C:\Users\User\Desktop\Data Science\Fall 2025\Generative AI\Project\genai-financial-doc-analysis\data\raw\all_stocks_long.csv")

print("Shape:", all_stocks.shape)
print("Columns:", all_stocks.columns.tolist())
print(all_stocks.head())

# Check unique tickers
print("Unique tickers:", all_stocks["Ticker"].unique())

# Convert Date column
all_stocks["Date"] = pd.to_datetime(all_stocks["Date"], errors="coerce")

print("Date range:", all_stocks["Date"].min(), "to", all_stocks["Date"].max())

# Check duplicates
dupes = all_stocks.duplicated(subset=["Date", "Ticker"]).sum()
print("Duplicate rows:", dupes)


Shape: (5020, 9)
Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Ticker']
                        Date        Open        High         Low       Close  \
0  2020-09-16 00:00:00-04:00  112.086293  112.835283  108.983321  109.070862   
1  2020-09-17 00:00:00-04:00  106.726601  109.138937  105.744154  107.329681   
2  2020-09-18 00:00:00-04:00  107.388051  107.854952  103.195633  103.925171   
3  2020-09-21 00:00:00-04:00  101.687903  107.183759  100.287187  107.076759   
4  2020-09-22 00:00:00-04:00  109.605857  109.780947  106.181894  108.759590   

      Volume  Dividends  Stock Splits Ticker  
0  154679000        0.0           0.0   AAPL  
1  178011000        0.0           0.0   AAPL  
2  287104900        0.0           0.0   AAPL  
3  195713800        0.0           0.0   AAPL  
4  183055400        0.0           0.0   AAPL  
Unique tickers: ['AAPL' 'MSFT' 'GOOG' 'AMZN']
Date range: 2020-09-16 00:00:00-04:00 to 2025-09-15 00:00:00-04:00
Duplicat

C:\Users\User\AppData\Local\Temp\ipykernel_18664\3830864899.py:14: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  all_stocks["Date"] = pd.to_datetime(all_stocks["Date"], errors="coerce")
